In [1]:
import numpy as np
import pandas as pd

In [2]:
leaked_df = pd.read_feather("../data/leak/leak.feather")

In [3]:
leaked_df.rename(columns={"meter_reading": "leaked_reading"}, inplace=True)

In [4]:
leaked_df.loc[leaked_df["leaked_reading"] < 0].head()

,building_id,meter,leaked_reading,timestamp


In [5]:
#leaked_df.loc[leaked_df["leaked_reading"] < 0, "leaked_reading"] = 0

In [6]:
leaked_df[leaked_df["building_id"] == 245].head()

,building_id,meter,leaked_reading,timestamp


In [7]:
#leaked_df = leaked_df[leaked_df["building_id"] != 245]

In [8]:
leaked_df["timestamp"] = leaked_df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [9]:
leaked_df.head()

,building_id,meter,leaked_reading,timestamp
0,0,0.0,0.0,2016-01-01 00:00:00
1,1,0.0,0.0,2016-01-01 00:00:00
2,2,0.0,0.0,2016-01-01 00:00:00
3,3,0.0,0.0,2016-01-01 00:00:00
4,4,0.0,0.0,2016-01-01 00:00:00


In [10]:
prediction_df = pd.read_csv("../submissions/submission-1.065.csv")

In [11]:
prediction_df.head()

,row_id,meter_reading
0,0,168.170537
1,1,60.459634
2,2,6.317182
3,3,233.037190
4,4,1094.490301


In [12]:
test_df = pd.read_csv("../data/raw/test.csv")

In [13]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [14]:
df = test_df.merge(prediction_df, on="row_id", how="left")
df = df.merge(leaked_df, on=["building_id", "meter", "timestamp"], how="left")
df["meter_reading"] = np.where(df["leaked_reading"].isna(), df["meter_reading"], df["leaked_reading"])

In [15]:
df.head()

,row_id,building_id,meter,timestamp,meter_reading,leaked_reading
0,0,0,0,2017-01-01 00:00:00,173.370293,173.370293
1,1,1,0,2017-01-01 00:00:00,53.512720,53.512720
2,2,2,0,2017-01-01 00:00:00,6.143042,6.143042
3,3,3,0,2017-01-01 00:00:00,101.701470,101.701470
4,4,4,0,2017-01-01 00:00:00,1141.240666,1141.240666


In [16]:
df.tail()

,row_id,building_id,meter,timestamp,meter_reading,leaked_reading
41697595,41697595,1444,0,2018-05-09 07:00:00,5.846339,NaN
41697596,41697596,1445,0,2018-05-09 07:00:00,3.844796,NaN
41697597,41697597,1446,0,2018-05-09 07:00:00,6.935650,NaN
41697598,41697598,1447,0,2018-05-09 07:00:00,153.462742,NaN
41697599,41697599,1448,0,2018-05-09 07:00:00,3.531959,NaN


In [17]:
df[["row_id", "meter_reading"]].to_csv("../submissions/submission-1.065-with-leaks.csv", index=False)